# Анализ резултатов работы прототипа OCR. Распознавание выполнено PaddleOCR 
- Автор: Кирилл Киселев
- Дата начала: 27.04.2022
- Описание исследования: анализ полученных результатов.

## Загрузка библиотек

In [1]:
import os
import pandas as pd
import cv2

## Глобальные переменные

In [2]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks/ocr


In [3]:
CSV_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results.csv'
# CSV_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_1.csv'
DATA_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/test_cases/'

## Загрузка и подготовка датасета

In [4]:
df = pd.read_csv(CSV_PATH, header=None)

In [5]:
df.columns = ['doc_name', 'barcode_number', 'doc_type', 'doc_number']
df.head()

,doc_name,barcode_number,doc_type,doc_number
0,devops-paddle-1 | {'id': 0,'barcode': '2810094793638','doc_type': None,'doc_number': '6010358884'}
1,devops-paddle-1 | {'id': 1,'barcode': '2860350795548','doc_type': 'Товарная накладная','doc_number': '6010498941'}
2,devops-paddle-1 | {'id': 2,'barcode': '2860350803625','doc_type': 'Товарная накладная','doc_number': '6010498985'}
3,devops-paddle-1 | {'id': 3,'barcode': '2860350826969','doc_type': 'Товарная накладная','doc_number': '6010503173'}
4,devops-paddle-1 | {'id': 4,'barcode': '2860350795340','doc_type': None,'doc_number': '6010498934'}


In [6]:
for col in df[['doc_name', 'barcode_number', 'doc_number']]:
    for i, _ in enumerate(df[col]):
        df[col][i] = df[col][i].split(' ')[-1]

In [7]:
for i, _ in enumerate(df['doc_number']):
    df['doc_number'][i] = df['doc_number'][i].split('}')[0]

df.head()

,doc_name,barcode_number,doc_type,doc_number
0,0,'2810094793638','doc_type': None,'6010358884'
1,1,'2860350795548','doc_type': 'Товарная накладная','6010498941'
2,2,'2860350803625','doc_type': 'Товарная накладная','6010498985'
3,3,'2860350826969','doc_type': 'Товарная накладная','6010503173'
4,4,'2860350795340','doc_type': None,'6010498934'


In [8]:
for col in df[['barcode_number', 'doc_type', 'doc_number']]:
    for i, _ in enumerate(df[col]):
        df[col][i] = df[col][i].replace("'", "")

In [9]:
for i, _ in enumerate(df['doc_type']):
    df['doc_type'][i] = df['doc_type'][i].split(' ', maxsplit=2)[-1]

In [10]:
df.head()

,doc_name,barcode_number,doc_type,doc_number
0,0,2810094793638,None,6010358884
1,1,2860350795548,Товарная накладная,6010498941
2,2,2860350803625,Товарная накладная,6010498985
3,3,2860350826969,Товарная накладная,6010503173
4,4,2860350795340,None,6010498934


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   doc_name        203 non-null    object
 1   barcode_number  203 non-null    object
 2   doc_type        203 non-null    object
 3   doc_number      203 non-null    object
dtypes: object(4)
memory usage: 6.5+ KB


In [12]:
for i, name in enumerate(df['doc_name']):
    if len(name) == 1:
        df['doc_name'][i] = '00' + df['doc_name'][i] + '.jpg'
    elif len(name) == 2:
        df['doc_name'][i] = '0' + df['doc_name'][i] + '.jpg'
    elif len(name) == 3:
        df['doc_name'][i] = df['doc_name'][i] + '.jpg'

In [13]:
df.head()

,doc_name,barcode_number,doc_type,doc_number
0,000.jpg,2810094793638,None,6010358884
1,001.jpg,2860350795548,Товарная накладная,6010498941
2,002.jpg,2860350803625,Товарная накладная,6010498985
3,003.jpg,2860350826969,Товарная накладная,6010503173
4,004.jpg,2860350795340,None,6010498934


## Визуализация

## Анализ данных

### Необходимые функции

In [14]:
def not_numeric_in_colunm(col):
    not_numeric = {}
    for i, value in enumerate(col):
        if col[i].isnumeric() == False:
            not_numeric[i] = value
    return not_numeric

In [15]:
def len_of_numbers(col):
    length_dict = {}
    for i, value in enumerate(col):
        if value.isnumeric():
            if len(value) not in length_dict.keys():
                length_dict[len(value)] = 1
            else:
                length_dict[len(value)] += 1
    return length_dict

In [16]:
some_dict = {1:2, 3:4}
some_dict.keys()

dict_keys([1, 3])

### Barcode

In [17]:
bad_barcode_df = df[df['barcode_number'] == 'None'].reset_index(drop=True)
bad_barcode_df

,doc_name,barcode_number,doc_type,doc_number
0,095.jpg,None,None,None
1,112.jpg,None,None,None
2,146.jpg,None,None,None
3,157.jpg,None,None,None
4,159.jpg,None,None,None


In [18]:
barcode_not_numeric = not_numeric_in_colunm(df['barcode_number'])
barcode_not_numeric

{95: 'None', 112: 'None', 146: 'None', 157: 'None', 159: 'None'}

In [19]:
length_of_barcode_numbers = len_of_numbers(df['barcode_number'])
length_of_barcode_numbers

{13: 198}

In [20]:
length_of_barcode_numbers[13] / len(df)

0.9753694581280788

In [21]:
bad_barcode_list = list(bad_barcode_df['doc_name'])

#### Вывод 
В столбце со штрих-кодами 4 пропущенных значения **None** для документов 95, 112, 146, 157, 159. Остальные значения соответствуют десятизначным штрих-кодам. Итого можно считать распознанными **97,5%** штрих-кодов.

### Doc type

In [22]:
df['doc_type'].unique()

array(['None', 'Товарная накладная', 'Товарная накладна',
       'Товарная накладнах', 'товарная накладная', 'Товарная наюладная',
       'Товаркая накладная', 'Товарная накладнвя', 'Товарная накпадная',
       'Товарная нанладная', 'Товарная макладная', 'товарная інакладная'],
      dtype=object)

In [23]:
df['doc_type'].value_counts()

Товарная накладная     109
None                    63
товарная накладная      15
Товарная накладна        6
Товарная накладнах       2
Товарная наюладная       2
Товаркая накладная       1
Товарная накладнвя       1
Товарная накпадная       1
Товарная нанладная       1
Товарная макладная       1
товарная інакладная      1
Name: doc_type, dtype: int64

In [24]:
1 - len(df[df['doc_type'] == 'None']) / len(df)

0.6896551724137931

In [25]:
bad_doc_type_df = df[df['doc_type'] == 'None']
bad_doc_type_df.head()

,doc_name,barcode_number,doc_type,doc_number
0,000.jpg,2810094793638,None,6010358884
4,004.jpg,2860350795340,None,6010498934
6,006.jpg,2860350795388,None,6010498935
7,007.jpg,2860350795418,None,6010498936
8,008.jpg,2860350803656,None,6010498987


In [26]:
bad_doc_type_list = list(bad_doc_type_df['doc_name'])
bad_doc_type_list

['000.jpg',
 '004.jpg',
 '006.jpg',
 '007.jpg',
 '008.jpg',
 '009.jpg',
 '020.jpg',
 '023.jpg',
 '027.jpg',
 '035.jpg',
 '036.jpg',
 '039.jpg',
 '042.jpg',
 '046.jpg',
 '048.jpg',
 '050.jpg',
 '054.jpg',
 '055.jpg',
 '062.jpg',
 '064.jpg',
 '072.jpg',
 '075.jpg',
 '090.jpg',
 '095.jpg',
 '102.jpg',
 '104.jpg',
 '105.jpg',
 '109.jpg',
 '110.jpg',
 '112.jpg',
 '115.jpg',
 '117.jpg',
 '119.jpg',
 '127.jpg',
 '130.jpg',
 '131.jpg',
 '133.jpg',
 '136.jpg',
 '141.jpg',
 '142.jpg',
 '145.jpg',
 '146.jpg',
 '148.jpg',
 '149.jpg',
 '152.jpg',
 '155.jpg',
 '157.jpg',
 '159.jpg',
 '163.jpg',
 '165.jpg',
 '166.jpg',
 '167.jpg',
 '168.jpg',
 '169.jpg',
 '170.jpg',
 '174.jpg',
 '179.jpg',
 '185.jpg',
 '186.jpg',
 '189.jpg',
 '190.jpg',
 '195.jpg',
 '202.jpg']

#### Вывод 
В столбце с типами документов 63 пропущенных значения **None**. Также в этом столбце есть почти распознанные данные, например **Товарная накладна**, **товарная інакладная** и другие. В почти распознанных данных допущено по 1 ошибке, так что, можно считать их проходящими по порогу распознавания. Итого качественно распознано 140 типов документов из 203 примеров, или всего **69,0%**.

### Doc number

In [27]:
bad_doc_number = df[df['doc_number'] == 'None']
bad_doc_number.head(10)

,doc_name,barcode_number,doc_type,doc_number
28,028.jpg,2810094889065,Товарная накладная,None
39,039.jpg,2860350669320,None,None
40,040.jpg,2810095177772,Товарная накладная,None
41,041.jpg,2860350669306,Товарная накладная,None
43,043.jpg,2860350693837,Товарная накладная,None
44,044.jpg,2860350653671,Товарная накладная,None
47,047.jpg,2860350653657,Товарная накладная,None
48,048.jpg,2860350617796,None,None
49,049.jpg,2860350627696,Товарная накладная,None
50,050.jpg,2810095176461,None,None


In [28]:
bad_doc_number_df = df[df['doc_number'].str.isnumeric() == False]
bad_doc_number_df.tail(10)

,doc_name,barcode_number,doc_type,doc_number
190,190.jpg,2860619145831,None,None
191,191.jpg,2860598926537,товарная накладная,None
193,193.jpg,2860598737003,товарная накладная,Konmueerue
195,195.jpg,2860615802998,None,None
196,196.jpg,2860613502609,Товарная накладная,None
197,197.jpg,2860532178794,Товарная накладная,None
198,198.jpg,2860593085420,товарная накладная,Konnyecrae
199,199.jpg,2860594030962,товарная накладная,None
200,200.jpg,2860597151404,Товарная накладная,Konwyoerao
202,202.jpg,2860601427549,None,None


In [29]:
len(bad_doc_number_df)

124

In [30]:
bad_doc_number_df['doc_number'].value_counts()

None          121
Konmueerue      1
Konnyecrae      1
Konwyoerao      1
Name: doc_number, dtype: int64

In [31]:
doc_number_not_numeric = not_numeric_in_colunm(df['doc_number'])
doc_number_not_numeric

{28: 'None',
 39: 'None',
 40: 'None',
 41: 'None',
 43: 'None',
 44: 'None',
 47: 'None',
 48: 'None',
 49: 'None',
 50: 'None',
 52: 'None',
 53: 'None',
 54: 'None',
 55: 'None',
 61: 'None',
 63: 'None',
 68: 'None',
 69: 'None',
 71: 'None',
 79: 'None',
 82: 'None',
 90: 'None',
 91: 'None',
 92: 'None',
 93: 'None',
 94: 'None',
 95: 'None',
 96: 'None',
 97: 'None',
 98: 'None',
 99: 'None',
 100: 'None',
 103: 'None',
 104: 'None',
 105: 'None',
 106: 'None',
 107: 'None',
 109: 'None',
 110: 'None',
 111: 'None',
 112: 'None',
 113: 'None',
 114: 'None',
 115: 'None',
 116: 'None',
 117: 'None',
 118: 'None',
 119: 'None',
 122: 'None',
 123: 'None',
 124: 'None',
 125: 'None',
 126: 'None',
 127: 'None',
 128: 'None',
 129: 'None',
 130: 'None',
 131: 'None',
 132: 'None',
 133: 'None',
 134: 'None',
 135: 'None',
 136: 'None',
 137: 'None',
 138: 'None',
 139: 'None',
 140: 'None',
 141: 'None',
 142: 'None',
 143: 'None',
 144: 'None',
 145: 'None',
 146: 'None',
 147: 'No

In [32]:
length_of_doc_numbers = len_of_numbers(df['doc_number'])
length_of_doc_numbers

{10: 79}

In [33]:
length_of_doc_numbers[10]/len(df)

0.3891625615763547

In [34]:
bad_doc_number_list = list(bad_doc_number_df['doc_name'])
bad_doc_number_list

['028.jpg',
 '039.jpg',
 '040.jpg',
 '041.jpg',
 '043.jpg',
 '044.jpg',
 '047.jpg',
 '048.jpg',
 '049.jpg',
 '050.jpg',
 '052.jpg',
 '053.jpg',
 '054.jpg',
 '055.jpg',
 '061.jpg',
 '063.jpg',
 '068.jpg',
 '069.jpg',
 '071.jpg',
 '079.jpg',
 '082.jpg',
 '090.jpg',
 '091.jpg',
 '092.jpg',
 '093.jpg',
 '094.jpg',
 '095.jpg',
 '096.jpg',
 '097.jpg',
 '098.jpg',
 '099.jpg',
 '100.jpg',
 '103.jpg',
 '104.jpg',
 '105.jpg',
 '106.jpg',
 '107.jpg',
 '109.jpg',
 '110.jpg',
 '111.jpg',
 '112.jpg',
 '113.jpg',
 '114.jpg',
 '115.jpg',
 '116.jpg',
 '117.jpg',
 '118.jpg',
 '119.jpg',
 '122.jpg',
 '123.jpg',
 '124.jpg',
 '125.jpg',
 '126.jpg',
 '127.jpg',
 '128.jpg',
 '129.jpg',
 '130.jpg',
 '131.jpg',
 '132.jpg',
 '133.jpg',
 '134.jpg',
 '135.jpg',
 '136.jpg',
 '137.jpg',
 '138.jpg',
 '139.jpg',
 '140.jpg',
 '141.jpg',
 '142.jpg',
 '143.jpg',
 '144.jpg',
 '145.jpg',
 '146.jpg',
 '147.jpg',
 '148.jpg',
 '149.jpg',
 '150.jpg',
 '151.jpg',
 '152.jpg',
 '153.jpg',
 '154.jpg',
 '155.jpg',
 '156.jpg',
 '15

#### Вывод 
В столбце с номерами документов 124 пропущенных значения. В них фигурируют либо **None** либо нечисловые значения, например **Konmueerue**. Качественно распознано 79 номеров документов из 203 примеров, или всего 38,9%.

## Результат исследования

In [35]:
bad_lists_union = sorted(list(set().union(bad_barcode_list, bad_doc_type_list, bad_doc_number_list)))

In [36]:
len(bad_lists_union)

142

In [37]:
len(df) - len(bad_lists_union)

61

In [38]:
1 -len(bad_lists_union) / len(df)

0.3004926108374384

## Общий вывод

Можно считать качественно распознанными лишь 61 документ из 203, что составляет **30%** из тестовых данных.